# Process Mesh file for sampling point clouds


In [2]:
import os
import numpy as np
import shutil

import shapefile

### 1. move all the mesh files into one folder

In [ ]:
mesh_folder_list = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/obj'

mesh_output_folder = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/mesh'

if not os.path.exists(mesh_output_folder):
    os.makedirs(mesh_output_folder)

for mesh_folder in os.listdir(mesh_folder_list):
    # print(mesh_folder)
    mesh_folder_path = os.path.join(mesh_folder_list, mesh_folder)
    for obj_folder in os.listdir(mesh_folder_path):
        # print(obj_folder)
        obj = os.path.join(mesh_folder_path, obj_folder)
        obj = os.path.join(obj, 'esriGeometryMultiPatch.obj')
        if not os.path.exists(obj):
            print(mesh_folder, obj_folder)
            continue
        destination = os.path.join(mesh_output_folder, mesh_folder.split('_')[0]+'_'+obj_folder.split('_')[1]+'.obj')
        # print(destination)
        # /media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/obj/V06_2016.gdb/OID_1/esriGeometryMultiPatch.obj
        shutil.copy(obj, destination)
        # break
    # break

### 2. Transform the local coordination to global coordination through wld3 file
![](Coordinate%20system.png)

In [2]:
# --------------------- Coordinate Transformation -------------------
def parse_wld3(wld3_path):
    with open(wld3_path, 'r') as f:
        lines = f.readlines()

    def parse_line(line):
        parts = line.strip().split()
        from_pt = np.array([float(x) for x in parts[0].split(',')])
        to_pt = np.array([float(x) for x in parts[1].split(',')])
        return from_pt, to_pt

    p1, g1 = parse_line(lines[0])
    p2, g2 = parse_line(lines[1])
    return p1, g1, p2, g2

def read_mesh(obj_file):
    vertices = []
    faces = []
    with open(obj_file, 'r') as f:
        for lines in f.readlines():
            parts = lines.strip().split(' ')
            if parts[0] == 'v':
                vertices.append(parts[1:4])
            elif parts[0] == 'f':
                face = []
                for p in parts[1:]:
                    face.append(int(p.split('/')[0]))
                faces.append(face)
    vertices = np.array(vertices, dtype=np.float64)
    # faces = np.array(faces, dtype=np.int32)
    return vertices, faces

def write_mesh(obj_file, vertices, faces):
    with open(obj_file, 'w') as f:
        f.write('# vertices: ' + str(len(vertices)) + '\n')
        f.write('# faces: ' + str(len(faces)) + '\n')
        f.write('\n')
        
        for v in vertices:
            f.write('v ' + str(v[0]) + ' ' + str(v[1]) + ' ' + str(v[2]) + '\n')
        
        for face in faces:
            f.write('f')
            for v_ind in face:
                f.write(' ' + str(v_ind))
            f.write('\n')

def coordinate_transformation(obj_file, wld3_file):
    # parse wld3_file
    p1, g1, p2, g2 = parse_wld3(wld3_file)
    
    # read mesh
    vertices, faces = read_mesh(obj_file)

    # Rotation Matrix
    R = np.array([
        [1, 0, 0],
        [0, 0, -1],
        [0, 1, 0]
    ])
    
    vertices = (R @ vertices.T).T + g1
    return vertices, faces


### 3. Generate Mesh with global coordinate and without material

In [33]:
mesh_folder_list = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/obj'

mesh_output_folder = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/mesh'

if not os.path.exists(mesh_output_folder):
    os.makedirs(mesh_output_folder)

for mesh_folder in os.listdir(mesh_folder_list):
    # print(mesh_folder)
    mesh_folder_path = os.path.join(mesh_folder_list, mesh_folder)
    for obj_folder in os.listdir(mesh_folder_path):
        # print(obj_folder)
        obj = os.path.join(mesh_folder_path, obj_folder, 'esriGeometryMultiPatch.obj')
        if not os.path.exists(obj):
            print(mesh_folder, obj_folder)
            continue
        destination = os.path.join(mesh_output_folder, mesh_folder.split('_')[0]+'_'+obj_folder.split('_')[1]+'.obj')
        # print(destination)
        
        # Coordinate Transformation
        wld3_path = os.path.join(mesh_folder_path, obj_folder, 'esriGeometryMultiPatch.wld3')
        vertices, faces = coordinate_transformation(obj, wld3_path)
        
        # Write Mesh
        write_mesh(destination, vertices, faces)
    #     break
    # break

V05_2016.gdb OID_1
V01_2016.gdb OID_1
V01_2016.gdb OID_3
V01_2016.gdb OID_2
V04_2016.gdb OID_11
VM12_2016.gdb OID_171
VM09_2016.gdb OID_581


### 4. Merge multiple mesh files into one mesh file

In [10]:
# Merge Mesh
def merge_mesh(mesh_folder, mesh_list, mesh_file):
    vertices_list = []
    faces_list = []

    faces_number = 0
    for mesh in mesh_list:
        mesh_path = os.path.join(mesh_folder, mesh)
        if not os.path.exists(mesh_path):
            print(mesh_path)
            continue
        vertices, faces = read_mesh(mesh_path)

        vertices_list += vertices.tolist()
        faces = [[x + faces_number for x in sublist] for sublist in faces]
        faces_list += faces
        faces_number += len(vertices)

    write_mesh(mesh_file, vertices_list, faces_list)

In [35]:

mesh_folder = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/mesh'
mesh_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal'

mesh_list = os.listdir(mesh_folder)
pre_index = 0
aft_index = 0
while True:
    pre_index = aft_index
    aft_index += len(mesh_list) // 10
    if aft_index > len(mesh_list):
        aft_index = len(mesh_list)
    print(pre_index, aft_index)

    merge_mesh(mesh_folder, mesh_list[pre_index:aft_index], os.path.join(mesh_file, 'Montreal_mesh_' + str(pre_index) + '.obj'))
    if aft_index == len(mesh_list):
        break

0 6313
6313 12626
12626 18939
18939 25252
25252 31565
31565 37878
37878 44191
44191 50504
50504 56817
56817 63130
63130 63131


### 5. Do a Test on Helios
Find a region of 1 * 4 km^2 (X * Y)

#### 5.1 Determine the range of scenes

In [58]:
mesh_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/Montreal_mesh_0.obj'

vertices, faces = read_mesh(mesh_file)

x_min, y_min, z_min = np.min(vertices, axis=0)
x_max, y_max, z_max = np.max(vertices, axis=0)

In [59]:
print(x_max-x_min)
print(y_max-y_min)
print(x_min, y_min, z_min)
print(x_max, y_max, z_max)

10935.784400000353
11662.650999999605
291931.3739999999 5032899.34 5.86040000000503
302867.15840000025 5044561.990999999 233.75389000000652


In [63]:
x_start = 291900 + (x_max - x_min) // 2
x_end = x_start + 2000

y_start = 5032800
y_end = y_start + 6000

In [66]:
# Merge Mesh
def merge_mesh_within_range(mesh_folder, mesh_file, x_start, x_end, y_start, y_end):
    vertices_list = []
    faces_list = []

    faces_number = 0
    for mesh in os.listdir(mesh_folder):
        mesh_path = os.path.join(mesh_folder, mesh)
        if not os.path.exists(mesh_path):
            print(mesh_path)
            continue
        vertices, faces = read_mesh(mesh_path)

        v0 = vertices[0]
        if not (x_start <= v0[0] <= x_end and y_start <= v0[1] <= y_end):
            continue

        vertices_list += vertices.tolist()
        faces = [[x + faces_number for x in sublist] for sublist in faces]
        faces_list += faces
        faces_number += len(vertices)

    write_mesh(mesh_file, vertices_list, faces_list)

In [65]:
mesh_folder = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/mesh'
mesh_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Montreal/Montreal_Helios_test.obj'

merge_mesh_within_range(mesh_folder, mesh_file, x_start, x_end, y_start, y_end)

In [67]:
print(x_start, x_end, y_start, y_end)

297367.0 299367.0 5032800 5038800


#### 5.2 Create shapefile as the line

In [1]:
x_flight_start = 297300
x_flight_end = 299400

y_flight_start = 5032800
y_flight_end = 5034800


In [3]:
# create
os.makedirs("/media/shangfeng/storage/shangfeng/BuildingWorld/BuildingWorld/Montreal/Montreal_Helios_shapefile_test", exist_ok=True)

# 创建 writer，指定类型为 POLYLINE
with shapefile.Writer("/media/shangfeng/storage/shangfeng/BuildingWorld/BuildingWorld/Montreal/Montreal_Helios_shapefile_test/test_lines", shapeType=shapefile.POLYLINE) as w:
    # add name
    w.field("Montreal", "C", size=20)

    x_list = []
    for x in range(int(x_flight_start+1360/2.0), x_flight_end, 1360):
        x_list.append(x)

    y_list = []
    for y in range(y_flight_start, y_flight_end, 1000):
        y_list.append(y)

    line = []
    for i, x in enumerate(x_list):
        print(i, x)
        if i % 2 == 0:
            for y in y_list:
                line.append([x, y])
        else:
            for y in reversed(y_list):
                print(y)
                line.append([x, y])

    lines = []
    lines.append(line)
    w.line(lines)
    w.record('Line')

    # # 添加第一条线（由两个点构成）
    # w.line([[[0, 0], [10, 10]]])  # 需要嵌套三层 list
    # w.record("Line 1")
    #
    # # 添加第二条线（由三个点构成）
    # w.line([[[20, 20], [25, 30], [30, 35]]])
    # w.record("Line B")


0 297980
1 299340
5033800
5032800


1